In [1]:
import pandas as pd
import numpy as np
import cv2 
import pytesseract

from imutils.object_detection import non_max_suppression
from pytesseract import Output

In [2]:
img = cv2.imread('../data/IMG_0014.jpeg')

In [3]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [4]:
gray = cv2.rotate(gray, cv2.ROTATE_90_COUNTERCLOCKWISE)

In [7]:
net = cv2.dnn.readNet('../models/frozen_east_text_detection.pb')

In [8]:
img_orig = img

(H, W) = img.shape[:2]
# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (32*(W//32), 32*(H//32))
rW = W / float(newW)
rH = H / float(newH)
# resize the image and grab the new image dimensions
img = cv2.resize(img, (newW, newH))
(H, W) = img.shape[:2]

In [9]:
layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]

In [10]:
blob = cv2.dnn.blobFromImage(img, 1.0, (W, H),
	(123.68, 116.78, 103.94), swapRB=True, crop=False)
#start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
#end = time.time()
# show timing information on text prediction
#print("[INFO] text detection took {:.6f} seconds".format(end - start))

In [11]:
(numRows, numCols) = scores.shape[2:4]
min_confidence = 0.5
rects = []
confidences = []
# loop over the number of rows
for y in range(0, numRows):
	# extract the scores (probabilities), followed by the geometrical
	# data used to derive potential bounding box coordinates that
	# surround text
	scoresData = scores[0, 0, y]
	xData0 = geometry[0, 0, y]
	xData1 = geometry[0, 1, y]
	xData2 = geometry[0, 2, y]
	xData3 = geometry[0, 3, y]
	anglesData = geometry[0, 4, y]

    # loop over the number of columns
	for x in range(0, numCols):
		# if our score does not have sufficient probability, ignore it
		if scoresData[x] < min_confidence:
			continue
		# compute the offset factor as our resulting feature maps will
		# be 4x smaller than the input image
		(offsetX, offsetY) = (x * 4.0, y * 4.0)
		# extract the rotation angle for the prediction and then
		# compute the sin and cosine
		angle = anglesData[x]
		cos = np.cos(angle)
		sin = np.sin(angle)
		# use the geometry volume to derive the width and height of
		# the bounding box
		h = xData0[x] + xData2[x]
		w = xData1[x] + xData3[x]
		# compute both the starting and ending (x, y)-coordinates for
		# the text prediction bounding box
		endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
		endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
		startX = int(endX - w)
		startY = int(endY - h)
		# add the bounding box coordinates and probability score to
		# our respective lists
		rects.append((startX, startY, endX, endY))
		confidences.append(scoresData[x])

In [12]:
boxes = non_max_suppression(np.array(rects), probs=confidences)
# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
	# scale the bounding box coordinates based on the respective
	# ratios
	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)
	# draw the bounding box on the image
	cv2.rectangle(img_orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
# show the output image
cv2.imshow("Text Detection", img_orig)
cv2.waitKey(0)

48

In [13]:
d = pytesseract.image_to_data(img, output_type=Output.DICT)
print(d.keys())

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [14]:
n_boxes = len(d['text'])
img2 = img
for i in range(n_boxes):
    if int(d['conf'][i]) > 20:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img2 = cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 3)

cv2.imshow('img', img2)
cv2.waitKey(0)

48